In [ ]:
!pip install --upgrade nibabel

Get data


In [ ]:
!wget "https://putpoznanpl-my.sharepoint.com/:u:/g/personal/dominik_pieczynski_put_poznan_pl/EWIZ_xm8wXpMjQDgF2VQ1csB4QuHPKoj5vDpj6CQi9p-AA?e=yQr6fn&download=1" -O public.zip
!unzip -q public.zip
!rm public.zip

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Create valid dirs

In [ ]:
!mkdir /content/FirstDataset/valid
!mkdir /content/SecondDataset/valid

In [ ]:
!mkdir /content/FirstDataset/train2

Checking the number of scans in dirs 

In [ ]:
import os
import shutil

print("FirstDataset")
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
print('train', len(filenames1))
print('test', len(os.listdir('/content/FirstDataset/test')))

print("SecondDataset")
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
print('train', len(filenames2))
print('test', len(os.listdir('/content/SecondDataset/test')))

FirstDataset
train 574
test 72
SecondDataset
train 100
test 25


In [ ]:
import os
import shutil
from pathlib import Path

# FirstDataset
filepath1 = Path('/content/FirstDataset/train')

first_X = []
first_y = []

for scan_path in sorted(filepath1.iterdir()):
  if scan_path.name.endswith('mask.nii.gz'):
    first_y.append(scan_path)
  else:
    first_X.append(scan_path)

print(len(first_X))
print(len(first_y))

# SecondDataset
filepath2 = Path('/content/SecondDataset/train')

second_X = []
second_y = []

for scan_path in sorted(filepath2.iterdir()):
    second_y.append(scan_path/'mask.nii.gz')
    second_X.append(scan_path/'T1w.nii.gz')

print(len(second_X))
print(len(second_y))



287
287
100
100


In [ ]:
from sklearn.model_selection import train_test_split

X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(first_X, first_y, train_size=0.9, random_state=42)
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(second_X, second_y, train_size=0.9, random_state=42)

X_train = X_train_1 + X_train_2
y_train = y_train_1 + y_train_2
X_valid = X_valid_1 + X_valid_2
y_valid = y_valid_1 + y_valid_2

print('X_train len:', len(X_train))
print('X_valid len:', len(X_valid))
print('y_train len:', len(y_train))
print('y_valid len:', len(y_valid))


X_train len: 348
X_valid len: 39
y_train len: 348
y_valid len: 39


###Eksport to png

Template methods

In [ ]:
import numpy as np
import nibabel as nib

from typing import Tuple, List
from pathlib import Path


def load_raw_volume(path: Path) -> Tuple[np.ndarray, np.ndarray]:
  data: nib.Nifti1Image = nib.load(str(path))
  data = nib.as_closest_canonical(data)
  raw_data = data.get_fdata(caching='unchanged', dtype=np.float32)
  return raw_data, data.affine


def load_labels_volume(path: Path) -> np.ndarray:
  return load_raw_volume(path)[0].astype(np.uint8)


def save_labels(data: np.ndarray, affine: np.ndarray, path: Path):
  nib.save(nib.Nifti1Image(data, affine), str(path))


def show_slices(slices: List[np.ndarray]):
   fig, axes = plt.subplots(1, len(slices))
   for i, data_slice in enumerate(slices):
       axes[i].imshow(data_slice.T, cmap="gray", origin="lower")

Create dirs for images

In [ ]:
!rm -rf /content/Images

!mkdir /content/Images

!mkdir /content/Images/valid
!mkdir /content/Images/valid/images
!mkdir /content/Images/valid/images/images
!mkdir /content/Images/valid/masks
!mkdir /content/Images/valid/masks/masks

!mkdir /content/Images/train
!mkdir /content/Images/train/images
!mkdir /content/Images/train/images/images
!mkdir /content/Images/train/masks
!mkdir /content/Images/train/masks/masks

print("done")

done


In [ ]:
!ls /content/Images/valid/images/images

In [ ]:
print(y_train)

Export to png

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np


image_train_path = '/content/Images/train/images/images/'
mask_train_path = '/content/Images/train/masks/masks/'

image_valid_path = '/content/Images/valid/images/images/'
mask_valid_path = '/content/Images/valid/masks/masks/'


print('X_train start')
for scan_idx, scan_path in enumerate(X_train):
  print(scan_path)

  raw_volume, affine = load_raw_volume(scan_path)

  for slice_idx in range(raw_volume.shape[0]):
    path = f'{image_train_path}train_image_{scan_idx}_slice_{slice_idx}_.png'

    volume = raw_volume[slice_idx]

    volume_max = np.amax(volume)
    volume_min = np.amin(volume)

    volume = (volume-volume_min)/(volume_max-volume_min)
    volume = volume*255
    volume = volume.astype('uint8')

    matplotlib.image.imsave(path, volume, cmap="gray")

print("X_train finish")


print('y_train start')
for scan_idx, scan_path in enumerate(y_train):
  print(scan_path)

  mask_volume = load_labels_volume(scan_path)
  for slice_idx in range(mask_volume.shape[0]):
    path = f'{mask_train_path}train_mask_{scan_idx}_slice_{slice_idx}_.png'

    matplotlib.image.imsave(path, mask_volume[slice_idx], cmap="gray")

print("y_train finish")


print('X_valid start')
for scan_idx, scan_path in enumerate(X_valid):
  print(scan_path)

  raw_volume, affine = load_raw_volume(scan_path)

  for slice_idx in range(raw_volume.shape[0]):
    path = f'{image_valid_path}valid_image_{scan_idx}_slice_{slice_idx}_.png'
    volume = raw_volume[slice_idx]

    volume_max = np.amax(volume)
    volume_min = np.amin(volume)

    volume = (volume-volume_min)/(volume_max-volume_min)
    volume = volume*255
    volume = volume.astype('uint8')

    matplotlib.image.imsave(path, volume, cmap="gray")
print("X_valid finish")


print('y_valid start')
for scan_idx, scan_path in enumerate(y_valid):
  print(scan_path)

  mask_volume = load_labels_volume(scan_path)
  for slice_idx in range(mask_volume.shape[0]):
    path = f'{mask_valid_path}train_mask_{scan_idx}_slice_{slice_idx}_.png'

    matplotlib.image.imsave(path, mask_volume[slice_idx], cmap="gray")
print("y_valid finish")



In [ ]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
from google.colab.patches import cv2_imshow

img = cv2.imread('/content/Images/valid/images/images/valid_image_2_slice_77_.png') 
  
print(img.shape)
print(np.amax(img))
print(np.amin(img))
cv2_imshow(img) 

# image = Image.open('/content/Images/train/images/images/train_image_0_slice_100_.png')#.convert('L')
# print(image.size)
# print(np.amax(image))
# print(np.amin(image))
# plt.imshow(image)

In [ ]:
img = cv2.imread('/content/Images/valid/masks/masks/train_mask_0_slice_105_.png') 
  
print(img.shape)
print(np.amax(img))
print(np.amin(img))
cv2_imshow(img) 

Save dataset

In [ ]:
!zip -r data_images2.zip /content/Images

In [ ]:
!cp /content/data_images2.zip -r /content/drive/MyDrive/ZPO_projekt